## INSTALL OpenVINO™ integration with TensorFlow


In [2]:
# Upload the required wheel files, models and images in a google drive folder
# Uncomment and run the below command to copy them in your current workspace
#!cp /content/drive/MyDrive/TF-OV/working_dir_files/* . 

!pip install --upgrade pip
!pip install pillow

# Install stock TensorFlow
!pip install tensorflow==2.4.1 

# Install OpenVINO™ integration with TensorFlow
!pip install openvino-tensorflow


In [3]:
!ldd --version

ldd (Ubuntu GLIBC 2.27-3ubuntu1.2) 2.27
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
Written by Roland McGrath and Ulrich Drepper.


In [4]:
!git clone https://github.com/openvinotoolkit/openvino_tensorflow.git
%cd openvino_tensorflow
!git submodule init
!git submodule update --recursive
%cd ..


Cloning into 'openvino_tensorflow'...
remote: Enumerating objects: 14188, done.
remote: Counting objects: 100% (965/965), done.
remote: Compressing objects: 100% (496/496), done.
remote: Total 14188 (delta 609), reused 756 (delta 458), pack-reused 13223
Receiving objects: 100% (14188/14188), 20.20 MiB | 31.15 MiB/s, done.
Resolving deltas: 100% (10901/10901), done.
/content/openvino_tensorflow
Submodule 'ocm' (https://github.com/intel/ocm) registered for path 'ocm'
Cloning into '/content/openvino_tensorflow/ocm'...
Submodule path 'ocm': checked out '3e699e473c26019970cd53ba4b32c30b820b6809'
/content


# Lets get the model

In [5]:
!curl -L "https://storage.googleapis.com/download.tensorflow.org/models/inception_v3_2016_08_28_frozen.pb.tar.gz" | tar -C openvino_tensorflow/examples/data -xz 


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84.5M  100 84.5M    0     0  62.9M      0  0:00:01  0:00:01 --:--:-- 62.8M


# Now lets infer

In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import backend as K
from IPython.display import HTML

import tensorflow as tf
import numpy as np
import time
import argparse
import matplotlib.pyplot as plt
import os
import openvino_tensorflow as ovtf
from subprocess import check_output, call


In [7]:
def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()

    assert os.path.exists(model_file), "Could not find directory"
    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

In [8]:

def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.io.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(
            file_reader, channels=3, name="png_reader")
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(
            tf.image.decode_gif(file_reader, name="gif_reader"))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
    else:
        image_reader = tf.image.decode_jpeg(
            file_reader, channels=3, name="jpeg_reader")
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.compat.v1.image.resize_bilinear(dims_expander,
                                                 [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.compat.v1.Session()
    result = sess.run(normalized)

    return result

In [9]:
def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

In [10]:
def infer_openvino_tensorflow(model_file, input_layer, output_layer, file_name , input_height, input_width, input_mean, input_std, label_file):
    print("CREATE MODEL - BEGIN")
    graph = load_graph(model_file)
    print("CREATE MODEL - END")

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)


    # update config params for openvino tensorflow
    config = tf.compat.v1.ConfigProto()
    config_ngraph_enabled = ovtf.update_config(config)

    print("PREDICTION - BEGIN") 

    with tf.compat.v1.Session(
            graph=graph, config=config_ngraph_enabled) as sess:
        t = read_tensor_from_image_file(
            file_name,
            input_height=input_height,
            input_width=input_width,
            input_mean=input_mean,
            input_std=input_std)
        # Warmup
        results = sess.run(output_operation.outputs[0],
                           {input_operation.outputs[0]: t})
        # Run
        
        for num_times in range(10):
            start = time.time()
            results = sess.run(output_operation.outputs[0],
                               {input_operation.outputs[0]: t})
            elapsed = time.time() - start
            print('Inference time in ms: %f' % (elapsed * 1000))
            
    print("PREDICTION - END")
    results = np.squeeze(results)

    if label_file:
        top_k = results.argsort()[-5:][::-1]
        labels = load_labels(label_file)
        for i in top_k:
            print(labels[i], results[i])
    else:
        print("No label file provided. Cannot print classification results")

In [11]:
%cd openvino_tensorflow/examples/data/

/content/openvino_tensorflow/examples/data


In [12]:
file_name = "grace_hopper.jpg"
model_file = "inception_v3_2016_08_28_frozen.pb"
label_file = "imagenet_slim_labels.txt"
input_height = 299
input_width = 299
input_mean = 0
input_std = 255
input_layer = "input"
output_layer = "InceptionV3/Predictions/Reshape_1"
backend_name = "CPU"
    

#Print list of available backends
print('Available Backends:')
backends_list = ovtf.list_backends()
for backend in backends_list:
    print(backend)
ovtf.set_backend(backend_name)

print("OpenVINO TensorFlow is enabled")
infer_openvino_tensorflow(model_file, input_layer, output_layer, file_name, input_height, input_width, input_mean, input_std, label_file )



Available Backends:
CPU
OpenVINO TensorFlow is enabled
CREATE MODEL - BEGIN
CREATE MODEL - END
PREDICTION - BEGIN
Inference time in ms: 169.880152
Inference time in ms: 160.679817
Inference time in ms: 164.356947
Inference time in ms: 163.445950
Inference time in ms: 166.368008
Inference time in ms: 160.454988
Inference time in ms: 173.298836
Inference time in ms: 161.949396
Inference time in ms: 164.052248
Inference time in ms: 162.285328
PREDICTION - END
military uniform 0.8343048
mortarboard 0.021869866
academic gown 0.010358192
pickelhaube 0.008008161
bulletproof vest 0.0053508896


In [13]:
#Disable
ovtf.disable()
print("")
print("")
print("OpenVINO TensorFlow is disabled")
infer_openvino_tensorflow(model_file, input_layer, output_layer, file_name, input_height, input_width, input_mean, input_std, label_file )
ovtf.enable()




OpenVINO TensorFlow is disabled
CREATE MODEL - BEGIN
CREATE MODEL - END
PREDICTION - BEGIN
Inference time in ms: 198.520184
Inference time in ms: 203.818321
Inference time in ms: 209.498405
Inference time in ms: 199.054480
Inference time in ms: 206.291437
Inference time in ms: 206.650734
Inference time in ms: 207.066536
Inference time in ms: 213.644743
Inference time in ms: 205.952406
Inference time in ms: 198.025465
PREDICTION - END
military uniform 0.83430624
mortarboard 0.021869497
academic gown 0.010358062
pickelhaube 0.008008163
bulletproof vest 0.0053508915
